# Classement des clients par montant total des paiements

Ce notebook analyse le classement des clients en fonction du montant total de leurs paiements sur la plateforme Olist.


In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

In [ ]:
# Load the customer payment rank data
data_path = 'reports/cte/customer_payment_rank.csv'
df = pd.read_csv(data_path)

print(f"Data loaded successfully from {data_path}")
print(f"Shape of the dataset: {df.shape}")

# Display the first 20 customers
df.head(20)

In [ ]:
# Basic statistics
print("Basic Statistics:")
df['total_payment_amount'].describe()

In [ ]:
# Plot the distribution of total payment amounts
plt.figure(figsize=(14, 7))
sns.histplot(data=df, x='total_payment_amount', bins=50, kde=True)
plt.title('Distribution du montant total des paiements par client')
plt.xlabel('Montant total des paiements (R$)')
plt.ylabel('Nombre de clients')
plt.xlim(0, 2000)
plt.show()

In [ ]:
# Top 20 customers by total payment
top_20 = df.head(20)

plt.figure(figsize=(14, 8))
sns.barplot(data=top_20, y='customer_unique_id', x='total_payment_amount')
plt.title('Top 20 clients par montant total des paiements')
plt.xlabel('Montant total des paiements (R$)')
plt.ylabel('Customer Unique ID')
plt.tick_params(axis='y', labelsize=8)
plt.show()

In [ ]:
# Cumulative distribution of payment amounts
df_sorted = df.sort_values('total_payment_amount', ascending=False)
df_sorted['cumulative_amount'] = df_sorted['total_payment_amount'].cumsum()
df_sorted['cumulative_percentage'] = (df_sorted['cumulative_amount'] / df_sorted['total_payment_amount'].sum()) * 100

plt.figure(figsize=(14, 7))
plt.plot(df_sorted['cumulative_percentage'])
plt.title('Distribution cumulative des paiements')
plt.xlabel('Nombre de clients')
plt.ylabel('Pourcentage cumulative des paiements')
plt.grid(True)
plt.show()

In [ ]:
# Number of customers contributing to 80% of total revenue (Pareto principle)
pareto_80 = df_sorted[df_sorted['cumulative_percentage'] <= 80].shape[0]
total_customers = df.shape[0]
percentage_80 = (pareto_80 / total_customers) * 100

print(f"Nombre de clients contribuant à 80% des revenus: {pareto_80}")
print(f"Pourcentage des clients: {percentage_80:.2f}%")

# Visualize Pareto principle
plt.figure(figsize=(14, 7))
plt.plot(df_sorted['cumulative_percentage'], label='Cumulative Payments')
plt.axhline(y=80, color='red', linestyle='--', label='80% of Revenue')
plt.axvline(x=pareto_80, color='orange', linestyle='--', label=f'{pareto_80} customers')
plt.title('Principle de Pareto - 80% des revenus')
plt.xlabel('Nombre de clients (triés par montant)')
plt.ylabel('Pourcentage cumulative des paiements')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Customers with zero payment
zero_payment = df[df['total_payment_amount'] == 0]
print(f"Nombre de clients avec un montant total de paiement de 0: {len(zero_payment)}")

zero_payment

In [ ]:
# Summary statistics by rank deciles
df['rank_decile'] = pd.qcut(df['payment_rank'], q=10, labels=False)
decile_stats = df.groupby('rank_decile')['total_payment_amount'].describe()

decile_stats

In [ ]:
# Box plots by deciles
plt.figure(figsize=(14, 8))
sns.boxplot(data=df, x='rank_decile', y='total_payment_amount')
plt.title('Distribution des montants par décile de classement')
plt.xlabel('Décile de classement')
plt.ylabel('Montant total des paiements (R$)')
plt.ylim(0, 500)
plt.show()

## Conclusion

Le classement des clients par montant total des paiements révèle des informations importantes:
- Le client avec le montant total de paiement le plus élevé a dépensé **13 664,08 R$**.
- Le montant moyen par client est de **166,60 R$**.
- Environ **20% des clients** contribuent à **80% des revenus** (principe de Pareto).
- Il y a **2 clients avec un montant total de paiement de 0 R$**.
